# Optimize your Docker Infrastructure with Python

## PyData NYC

November 11, 2015

Ryan J. O'Neil  
<ryanjoneil@gmail.com>  

### Who's this guy?

By Day: Lead Engineer @ Yhat, Inc.  
By Night: PhD Candidate in SEOR @ George Mason University

Formerly:
* Simulation, modeling, optimization @ MITRE
* Data Journo @ The Washington Post

Not a data scientist.

### Motivation

Consider the DevOps Engineer.

#### It's a noble group...

![](images/scotty.jpg)

### ...with grave responsibilities.

One of the important functions of DevOps is the creation of environments for:

* Software development
* Testing & quality assurance
* Running operational systems
* Whatever else you might need a computing environment for

#### There's even a Venn diagram about DevOps.

And we all know how much we like Venn diagrams.

![](images/devops-venn.svg)

#### A typical DevOps function

Say you work on _Yet Another Enterprise Java Application (TM)_. YAEJA generates lots of \$\$\$\$ for _Yet Another Enterprise Software Company_ and it keeps you gainfully employed!

Say you need to upgrade portions of the system YAEJA it runs _(and depends)_ on.

Youo don't want to just upgrade the production instances without testing first. So you need two environments from your DevOps person.

One for running the production instance:

|     | Command                                                        |
|-----|----------------------------------------------------------------|
| $A$ | Install the Java compiler and runtime environment.             |
| $B$ | Download a set of external dependencies.                       |
| $C$ | Set up a an EntepriseDB (TM) schema and populate it with data. |

And another for testing it with the new version of the utility:

|     | Command                                                        |
|-----|----------------------------------------------------------------|
| $A$ | Install the Java compiler and runtime environment.             |
| $B$ | Download a set of external dependencies.                       |
| $C$ | Set up a an EntepriseDB (TM) schema and populate it with data. |
| $D$ | Update the underlying system utility.                          |

If _Yet Another Enterprise Java Application (TM)_ integrates with a number of optional third party applications, you might need an additional test environment for each one.

If those interact in interesting ways, you may need test environments for different combinations of third party integrations.

#### Looks like your DevOps person will be working all weekend

![](images/saturday.png)

#### In the old days...

At a big software shop, devops may be responsible for the continual setup and teardown of dozens (or _hundreds_) of system configurations for a single software project.

This used to be done with physical hardware on _(sometimes)_ fresh operating system installs.

Most medium-to-large software shops had an air conditioned room that looked like this.

![](images/cable-spaghetti.jpg)

#### I'm dating myself here

If you had to reproduce an environment, you had a few options:

* Start with a fresh install of the operating system
* Save the results of your setup to a CD (?!?!) and load that onto a box
* Hope that uninstalling and resintalling the relevant components is good enough
    + A _lot_ of people did this
    + It's a _really_ bad idea
    + Most software leaves behind relics
        - Logs...
        - Data files...
        - Configuration...

### What's a container?

Docker is so hot right now.

#### Disclaimer

Nothing in this section is actually new.

Container architectures have been around since `chroot` jails in V7 Unix.

But now they're so convenient they feel _(and are raising venture capital)_ that way!

#### On second though, let's have that history lesson

This might go back to the paleolithic era, but Wikipedia isn't aware of it.

* 1979: `chroot` jails added to System 7 Unix at Bell Labs
    + Last version of Unix before it was commercializated by AT&T
    + Ran on a DEC PDP-11 minicomputer
* 1982: Bill Joy ports `chroot` jails in BSD
    + 
* 

#### Different kinds of virtualization

I don't actually know much about this but it sounds cool.

* Hardware level
    
* Software level

### Docker cache mechanics

Maybe I should call this section UnionFS mechanics, but then Docker is so hot right now.

#### A Tale of Two Dockerfiles

Two similar dockerfiles here (ABC

#### What happens when build these into images?

#### 

#### 

#### 

#### 

### Problem statement

Let's write this thing in LaTeX!

### Partitioning sets with PuLP

We'll start with a really easy NP-Complete problem...

### Finding maximal cliques with NetworkX

...add another NP-Complete subproblem...

### Model construction

...and then we'll solve both of them.

### Results

No, really. That's how we do this thing.